# Data Cleaning 

In [1]:
using Pkg;

# Installing packages
#Pkg.add("DelimitedFiles")
#Pkg.add("CSV")
#Pkg.add("DataFrames")
#Pkg.add("Pandas")
#Pkg.add("CategoricalArrays")
#Pkg.add("DataFramesMeta")
#Pkg.add("ScikitLearn")

In [2]:
using DelimitedFiles
using CSV
using Statistics 
using DataFramesMeta #using DataFrames
using CategoricalArrays;
include("./utils/preprocessing_utils.jl")

quartilOutliers (generic function with 1 method)

In [3]:
data = CSV.File("./dataset/music_genre.csv") |> DataFrame;
first(data,10)

Row,Artist Name,Track Name,Popularity,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_in min/ms,time_signature,Class
,String,String,Float64?,Float64,Float64,Float64?,Float64,Int64,Float64,Float64,Float64?,Float64,Float64,Float64,Float64,Int64,Int64
1,Bruno Mars,That's What I Like (feat. Gucci Mane),60.0,0.854,0.564,1.0,-4.964,1,0.0485,0.0171,missing,0.0849,0.899,134.071,234596.0,4,5
2,Boston,Hitch a Ride,54.0,0.382,0.814,3.0,-7.23,1,0.0406,0.0011,0.00401,0.101,0.569,116.454,251733.0,4,10
3,The Raincoats,No Side to Fall In,35.0,0.434,0.614,6.0,-8.334,1,0.0525,0.486,0.000196,0.394,0.787,147.681,109667.0,4,6
4,Deno,Lingo (feat. J.I & Chunkz),66.0,0.853,0.597,10.0,-6.528,0,0.0555,0.0212,missing,0.122,0.569,107.033,173968.0,4,5
5,Red Hot Chili Peppers,Nobody Weird Like Me - Remastered,53.0,0.167,0.975,2.0,-4.279,1,0.216,0.000169,0.0161,0.172,0.0918,199.06,229960.0,4,10
6,The Stooges,Search and Destroy - Iggy Pop Mix,53.0,0.235,0.977,6.0,0.878,1,0.107,0.00353,0.00604,0.172,0.241,152.952,208133.0,4,6
7,Solomon Burke,None Of Us Are Free,48.0,0.674,0.658,5.0,-9.647,0,0.104,0.404,1.34e-6,0.0981,0.677,143.292,329387.0,4,2
8,Randy Travis,On the Other Hand,55.0,0.657,0.415,5.0,-9.915,1,0.025,0.175,5.65e-6,0.132,0.347,96.03,3.10578,4,4
9,Professional Murder Music,Slow,29.0,0.431,0.776,10.0,-5.403,1,0.0527,2.21e-5,0.0013,0.179,0.318,120.857,237867.0,4,8


## Data Preprocessing

1. Set Data Types

In [48]:
df = copy(data);

df."key" = map((x) -> toString(x), df."key")
df."mode" = map((x) -> toString(x), df."mode")
df."time_signature" = map((x) -> toString(x), df."time_signature")
df."Class" = map((x) -> toString(x), df."Class")

categorical_columns = ["Artist Name","Track Name","key","mode","Class","time_signature"]

for category in categorical_columns
    df[:,category] = categorical(df[:,category]);
end

describe(df)

Row,variable,mean,min,median,max,nmissing,eltype
,Symbol,Union…,Any,Union…,Any,Int64,Type
1,Artist Name,,$NOT,,◊†◊ê◊ì◊î,0,String
2,Track Name,,"""45""",,◊™◊™◊ó◊†◊†◊ô ◊ê◊ú◊ô◊ô,0,String
3,Popularity,44.5121,1.0,44.0,100.0,428,"Union{Missing, Float64}"
4,danceability,0.543433,0.0596,0.545,0.989,0,Float64
5,energy,0.662777,2.03e-5,0.7,1.0,0,Float64
6,key,,1.0,,9.0,2014,"Union{Missing, String}"
7,loudness,-7.91066,-39.952,-7.016,1.355,0,Float64
8,mode,,0,,1,0,String
9,speechiness,0.079707,0.0225,0.0474,0.955,0,Float64


In [49]:

for colname in names(select(df, Not(categorical_columns)))
    noNa_column = skipmissing(df[:,colname])
    (sup_outlier,inf_outlier,ext_sup_outlier,ext_inf_outlier) = quartilOutliers(noNa_column)
    println("----",colname,"----")
    
    sup_outliers = findall(x-> x > sup_outlier, noNa_column)
    inf_outliers = findall(x-> x < inf_outlier, noNa_column)
    
    ext_sup_outliers = findall(x-> x > ext_sup_outlier, noNa_column)
    ext_inf_outliers = findall(x-> x < ext_inf_outlier, noNa_column)

    println("Outliers inf (x<",round(inf_outlier; digits=3),"):",length(inf_outliers)-length(ext_inf_outliers)," sup (x>",round(sup_outlier; digits=3),"):",length(sup_outliers)-length(ext_sup_outliers))
    println("Extreme outliers inf (x<",round(ext_inf_outlier; digits=3),"):",length(ext_inf_outliers)," sup (x>",round(ext_sup_outlier; digits=3),"):",length(ext_sup_outliers))

    if ((length(ext_inf_outliers)+length(ext_sup_outliers))>0)
        allowmissing!(df, colname);
        df[ext_sup_outliers,colname] .= missing
        df[ext_inf_outliers,colname] .= missing
    end
end

----Popularity----
Outliers inf (x<-1.5):0 sup (x>90.5):41
Extreme outliers inf (x<-36.0):0 sup (x>125.0):0
----danceability----
Outliers inf (x<0.091):31 sup (x>1.0):0
Extreme outliers inf (x<-0.249):0 sup (x>1.34):0
----energy----
Outliers inf (x<-0.017):0 sup (x>1.386):0
Extreme outliers inf (x<-0.544):0 sup (x>1.913):0
----loudness----
Outliers inf (x<-16.062):634 sup (x>1.335):3
Extreme outliers inf (x<-22.585):181 sup (x>7.858):0
----speechiness----
Outliers inf (x<-0.038):0 sup (x>0.155):803
Extreme outliers inf (x<-0.11):0 sup (x>0.228):1231
----acousticness----
Outliers inf (x<-0.64):0 sup (x>1.079):0
Extreme outliers inf (x<-1.285):0 sup (x>1.723):0
----instrumentalness----
Outliers inf (x<-0.3):0 sup (x>0.5):1020
Extreme outliers inf (x<-0.6):0 sup (x>0.8):1388
----liveness----
Outliers inf (x<-0.143):0 sup (x>0.499):642
Extreme outliers inf (x<-0.384):0 sup (x>0.74):314
----valence----
Outliers inf (x<-0.266):0 sup (x>1.235):0
Extreme outliers inf (x<-0.828):0 sup (x>1.797)

In [50]:
describe(df)

Row,variable,mean,min,median,max,nmissing,eltype
,Symbol,Union…,Any,Union…,Any,Int64,Type
1,Artist Name,,$NOT,,◊†◊ê◊ì◊î,0,String
2,Track Name,,"""45""",,◊™◊™◊ó◊†◊†◊ô ◊ê◊ú◊ô◊ô,0,String
3,Popularity,44.5121,1.0,44.0,100.0,428,"Union{Missing, Float64}"
4,danceability,0.543433,0.0596,0.545,0.989,0,Float64
5,energy,0.662777,2.03e-5,0.7,1.0,0,Float64
6,key,,1.0,,9.0,2014,"Union{Missing, String}"
7,loudness,-7.72315,-22.557,-6.981,1.355,181,"Union{Missing, Float64}"
8,mode,,0,,1,0,String
9,speechiness,0.0608453,0.0225,0.045,0.227,1231,"Union{Missing, Float64}"


### Quantity of missing values for each column

In [51]:
mvalues = describe(df,:nmissing,:eltype,:mean,:std,:min,:max)
mvalues.prop = mvalues.nmissing./sum(mvalues.nmissing)*100
mvalues = mvalues[mvalues.nmissing.>0,:]

Row,variable,nmissing,eltype,mean,std,min,max,prop
,Symbol,Int64,Type,Union…,Union…,Any,Any,Float64
1,Popularity,428,"Union{Missing, Float64}",44.5121,17.4269,1.0,100.0,4.23469
2,key,2014,"Union{Missing, String}",,,1.0,9.0,19.9268
3,loudness,181,"Union{Missing, Float64}",-7.72315,3.59986,-22.557,1.355,1.79084
4,speechiness,1231,"Union{Missing, Float64}",0.0608453,0.0403151,0.0225,0.227,12.1797
5,instrumentalness,5765,"Union{Missing, Float64}",0.0974663,0.199419,1.0e-6,0.799,57.0397
6,liveness,314,"Union{Missing, Float64}",0.184099,0.131646,0.0119,0.739,3.10676
7,duration_in min/ms,174,"Union{Missing, Float64}",1.96123e5,1.00199e5,0.50165,510587.0,1.72158


### Missing values Imputation

#### Correlation analysis

In [75]:
na_numeric_columns = ["Popularity","loudness","speechiness","instrumentalness","liveness","duration_in min/ms"]

df_numerical_columns = select(df[completecases(df),:], Not(categorical_columns))

table_names = append!(["col_name"],na_numeric_columns)

corr_matrix = DataFrame([ name =>[] for name in table_names])
for i in 1:size(df_numerical_columns,2)
    push!(corr_matrix,zeros(length(table_names)))
end

for (index,column) in enumerate(names(df_numerical_columns))
    corr_matrix[index,"col_name"] = column
    for sub_column in na_numeric_columns
        corr_value = cor(df_numerical_columns[:,sub_column],df_numerical_columns[:,column])
        if (abs(corr_value)>0.5)
            corr_matrix[index,sub_column] = corr_value
        end
    end
end

corr_matrix

Row,col_name,Popularity,loudness,speechiness,instrumentalness,liveness,duration_in min/ms
,Any,Any,Any,Any,Any,Any,Any
1,Popularity,1.0,0.0,0.0,0.0,0.0,0.0
2,danceability,0.0,0.0,0.0,0.0,0.0,0.0
3,energy,0.0,0.760718,0.0,0.0,0.0,0.0
4,loudness,0.0,1.0,0.0,0.0,0.0,0.0
5,speechiness,0.0,0.0,1.0,0.0,0.0,0.0
6,acousticness,0.0,-0.577811,0.0,0.0,0.0,0.0
7,instrumentalness,0.0,0.0,0.0,1.0,0.0,0.0
8,liveness,0.0,0.0,0.0,0.0,1.0,0.0
9,valence,0.0,0.0,0.0,0.0,0.0,0.0


In [8]:
using ScikitLearn
#@sk_import neighbors: KNeighborsClassifier
@sk_import neighbors: KNeighborsRegressor
@sk_import linear_model:LogisticRegression
using ScikitLearn.CrossValidation: cross_val_score

In [56]:
df_numerical_columns = select(df[completecases(df),:], Not(categorical_columns))
X = select(df_numerical_columns,Not(["Popularity","instrumentalness"]))
y1 = select(df_numerical_columns,"Popularity")
y2 = select(df_numerical_columns,"instrumentalness")

model = KNeighborsRegressor(n_neighbors=5)
model.fit(Matrix(X), Matrix(y1))

PyObject KNeighborsRegressor()

In [57]:
df_numerical_columns = select(df, Not(categorical_columns))

df_instrum = select(df_numerical_columns,Not(["Popularity"]))
df_instrum = df_instrum[completecases(df_instrum).==0,:]

df_popular = select(df_numerical_columns,Not(["instrumentalness"]))
df_popular = df_popular[completecases(df_popular).==0,:]

Row,Popularity,danceability,energy,loudness,speechiness,acousticness,liveness,valence,tempo,duration_in min/ms
,Float64?,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64
1,missing,0.548,0.711,-8.44,0.107,0.542,0.355,0.618,119.675,4.01563
2,missing,0.618,0.438,-7.495,0.0762,0.407,0.5,0.651,155.014,209307.0
3,missing,0.523,0.441,-9.928,0.0311,0.465,0.365,0.49,171.246,3.38512
4,missing,0.381,0.214,-11.629,0.0337,0.944,0.106,0.393,127.581,3.586
5,missing,0.545,0.919,-7.679,0.0483,0.00269,0.0978,0.386,104.364,313573.0
6,missing,0.521,0.527,-6.371,0.0296,0.0931,0.106,0.273,122.271,259627.0
7,missing,0.416,0.623,-7.898,0.045,0.121,0.27,0.194,130.175,311625.0
8,missing,0.753,0.591,-7.091,0.045,0.0741,0.108,0.563,101.941,2.74222
9,missing,0.368,0.274,-14.878,0.0413,0.956,0.0658,0.0459,84.549,4.5


In [61]:
names(df_popular)

10-element Vector{String}:
 "Popularity"
 "danceability"
 "energy"
 "loudness"
 "speechiness"
 "acousticness"
 "liveness"
 "valence"
 "tempo"
 "duration_in min/ms"

In [129]:
model.predict(collect(predictor_df[1,:]))

LoadError: PyError ($(Expr(:escape, :(ccall(#= C:\Users\mhorm\.julia\packages\PyCall\ygXW2\src\pyfncall.jl:43 =# @pysym(:PyObject_Call), PyPtr, (PyPtr, PyPtr, PyPtr), o, pyargsptr, kw))))) <class 'ValueError'>
ValueError('Expected 2D array, got 1D array instead:\narray=[ 8.54000e-01  5.64000e-01 -4.96400e+00  4.85000e-02  1.71000e-02\n  8.49000e-02  8.99000e-01  1.34071e+02  2.34596e+05].\nReshape your data either using array.reshape(-1, 1) if your data has a single feature or array.reshape(1, -1) if it contains a single sample.')
  File "C:\Users\mhorm\.julia\conda\3\lib\site-packages\sklearn\neighbors\_regression.py", line 229, in predict
    neigh_ind = self.kneighbors(X, return_distance=False)
  File "C:\Users\mhorm\.julia\conda\3\lib\site-packages\sklearn\neighbors\_base.py", line 745, in kneighbors
    X = self._validate_data(X, accept_sparse="csr", reset=False, order="C")
  File "C:\Users\mhorm\.julia\conda\3\lib\site-packages\sklearn\base.py", line 577, in _validate_data
    X 

In [58]:
pred_popular = model.predict(Matrix(select(df_popular,Not("Popularity"))))

428×1 Matrix{Float64}:
 36.6
 57.6
 28.2
 37.0
 22.4
 48.0
 33.8
 53.4
 32.8
 37.6
 21.2
 41.4
 36.0
  ⋮
 34.6
 48.8
 38.6
 38.8
 49.6
 46.8
 53.8
 47.0
 33.4
 44.8
 33.0
 28.4

In [55]:
println("Size: ",size(pred_popular))
println("Type:",typeof(pred_popular))

Size: (428, 1)
Type:Matrix{Float64}


In [69]:
na_numeric_columns = ["Popularity","instrumentalness"]
predictor_columns = [ "danceability","energy","loudness","speechiness","acousticness","liveness","valence","tempo","duration_in min/ms"]
predictor_df = select(df,Not(predictor_columns))

for na_column in na_numeric_columns
    for index in 1:size(df,1)
        if (ismissing(df[index,na_numeric_columns]))
            print(df[index,na_numeric_columns])
            #df[index,na_column] = model.predict(Matrix(predictor[index,:]))
        end
    end
end


In [146]:
v = collect(predictor_df[1,:])
v_matrix = permutedims(reshape(v, length(v), 1))
model.predict(v_matrix)[1]

47.2

In [151]:
na_numeric_columns = ["Popularity","instrumentalness"]
predictor_columns = [ "danceability","energy","loudness","speechiness","acousticness","liveness","valence","tempo","duration_in min/ms"]
predictor_df = select(df,predictor_columns)

for na_column in na_numeric_columns
    for (index,row) in enumerate(completecases(df))
        if (row==false)
            v = collect(predictor_df[index,:])
            v_matrix = permutedims(reshape(v, length(v), 1))
            df[index,na_column] = model.predict(v_matrix)[1]
        end
    end
end

In [152]:
describe(df)

Row,variable,mean,min,median,max,nmissing,eltype
,Symbol,Union…,Any,Union…,Any,Int64,Type
1,Artist Name,,$NOT,,◊†◊ê◊ì◊î,0,String
2,Track Name,,"""45""",,◊™◊™◊ó◊†◊†◊ô ◊ê◊ú◊ô◊ô,0,String
3,Popularity,43.2799,1.0,43.0,98.0,0,"Union{Missing, Float64}"
4,danceability,0.543433,0.0596,0.545,0.989,0,Float64
5,energy,0.662777,2.03e-5,0.7,1.0,0,Float64
6,key,,1.0,,9.0,2014,"Union{Missing, String}"
7,loudness,-7.91066,-39.952,-7.016,1.355,0,Float64
8,mode,,0,,1,0,String
9,speechiness,0.079707,0.0225,0.0474,0.955,0,Float64


In [70]:
size(df,1)

17996

### Duplicate rows

In [ ]:
# Duplicates (without names columns)
println("We have found ",size(df[:,3:end],1)-size(unique(df[:,3:end]),1)," duplicates rows.")
df = unique(df[:,3:end]);